In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from VirtualWorld.World import World
from Knowledge.Article import Article
from Knowledge.Source import Source

In [3]:
world = World()

Big Bang!


ValueError: empty range for randrange() (1,1, 0)

In [ ]:
topic = world.topics[63]
world.knowledge.filter_articles_by_topic(topic)

In [ ]:
from NPC.Person import Person

In [ ]:
Person.living_population

In [ ]:
world.towns['Raleigh'].citizens[10].house_number

In [ ]:
world.towns['Raleigh'].citizens[10].name

In [ ]:
world.population

In [ ]:
for house in world.towns['Raleigh'].homes.values(): 
    print(house['family'], house['occupants'])